In [1]:
#importing the important package for the model

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from nltk import WordNetLemmatizer,word_tokenize,PorterStemmer
from nltk.corpus import stopwords
import sklearn 
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import pickle

In [2]:
#Reading the Raw data from spotify datset 
rwdata=pd.read_csv("spotify_millsongdata.csv")

Understanding the values in  dataset
like - Null values,Duplicate values,unique value and shape

In [3]:
#Info dataset 
rwdata.info()


#There is 4 columns and all are in string type

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 57650 entries, 0 to 57649
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   artist  57650 non-null  object
 1   song    57650 non-null  object
 2   link    57650 non-null  object
 3   text    57650 non-null  object
dtypes: object(4)
memory usage: 1.8+ MB


In [4]:
#Find the shape of the dataset
print(f'Row :',rwdata.shape[0])
print(f'Columns :',rwdata.shape[1])

Row : 57650
Columns : 4


In [5]:
#Finding the null values
rwdata.isnull().sum().sort_values(ascending=False)

artist    0
song      0
link      0
text      0
dtype: int64

In [6]:
#Finding the Unique values
rwdata.nunique().sort_values(ascending=False)

link      57650
text      57494
song      44824
artist      643
dtype: int64

In [7]:
#Finding the duplicate values 
rwdata.duplicated().sum()

0

In [8]:
#droping the unwanted columns
rwdata=rwdata.drop(columns=['link'],axis=1).reset_index(drop=True)

In [9]:
rwdata['artist']

0                ABBA
1                ABBA
2                ABBA
3                ABBA
4                ABBA
             ...     
57645    Ziggy Marley
57646    Ziggy Marley
57647            Zwan
57648            Zwan
57649            Zwan
Name: artist, Length: 57650, dtype: object

In [10]:
#As dataset has 57650 data. Let make it to smaller to get faster output
spotiy=rwdata.sample(3892)

#I have write th dataset to my local dise
#spotiy.to_csv("spotiy.csv",index=0)

In [11]:
# import the csv file # It also change the index to 0-9
spotiy=pd.read_csv('spotiy.csv')

In [12]:
spotiy

,artist,song,text
0,Ne-Yo,Tell Me A Secret,"I got 5 kisses, where you want 'em at? \r\nEi..."
1,Bread,The Last Time,"Hey, didn't you believe me when I told you I w..."
2,Gino Vannelli,Words Can Kill,Why must we fight \r\nMust we shatter sweet d...
3,Ufo,Hot 'n' Ready,You knock me out you're like a well oiled Smit...
4,Eric Clapton,Have You Ever Loved A Woman,Have you ever loved a woman \r\nSo much you t...
...,...,...,...
3887,Hank Snow,Broken Hearted,"You ask me why I'm broken hearted \r\nWhy, I'..."
3888,Alice Cooper,Dragontown,"Well, here you are lying bleeding on a grimy s..."
3889,Michael Bolton,At Last,At last \r\nMy love has come along \r\nMy lo...
3890,Ozzy Osbourne,Changes,I feel unhappyi am so sadi'v lost the best fri...


In [13]:
spotiy.text

0       I got 5 kisses, where you want 'em at?  \r\nEi...
1       Hey, didn't you believe me when I told you I w...
2       Why must we fight  \r\nMust we shatter sweet d...
3       You knock me out you're like a well oiled Smit...
4       Have you ever loved a woman  \r\nSo much you t...
                              ...                        
3887    You ask me why I'm broken hearted  \r\nWhy, I'...
3888    Well, here you are lying bleeding on a grimy s...
3889    At last  \r\nMy love has come along  \r\nMy lo...
3890    I feel unhappyi am so sadi'v lost the best fri...
3891    When they found you, you were driven and hungr...
Name: text, Length: 3892, dtype: object

Text Pre-processing for the dataset

In [14]:
spotiy['text']=spotiy['text'].str.lower().replace(r'^\w\s',' ').replace(r'\n',' ',regex=True)

Stemming for text - which is used to normalize the word in the text

In [15]:
stem=PorterStemmer()

def token(text):
    token=word_tokenize(text)
    stemword=[stem.stem(i) for i in token]
    return " ".join(stemword)    

In [16]:
# use lambda funcation to apply this funcation in all the text dataset
spotiy['text']=spotiy['text'].apply(lambda x: token(x))

##### Now we are going to convert the word into numbers. Word to vectorizer 
##### myself i use TF-IDF.
###### We also has Keras one-hot encoding,BoW,Word2vec 

In [17]:
# Fitting the text into vectorizer 
vector=TfidfVectorizer(analyzer="word",stop_words="english")
t2n=vector.fit_transform(spotiy['text'])


In [18]:
spotiy['text'][300]

"( choru ) pay no attent to alic , she 's drunk all the time , hook on that wine , bunch of it , and it ruin her mind . pay no attent to alic , they say she 's a sot , sane she is not , but she love it , and it 's all she 's got . she made that appl pie from a memori , made them biscuit from a recollect that she had . she cook that chicken too long but she do n't know that , oh what the hell , it ai n't too bad . ( choru ) do n't talk about the war , i wa a coward , talk about fish and all the good time raisin ' hell . empti that one down , we 'll get anoth one , it 's get late , you might as well . though we ram your car into a ditch , man do n't sweat it , i know ben down at the shell station and he 'll get it out . alic , put your ash in that ashtray , i swear woman , you 're gon na burn down the hous . pay no attent to alic , she 's drunk all the time ..."

Cosine similarity is use to find the similary between 2 varible 

In [19]:
# Fit in to cosine which is used to find the similarity between 2 variable 
songsim=cosine_similarity(t2n)
songsim[0]

array([1.        , 0.18604004, 0.03459491, ..., 0.01582367, 0.01732394,
       0.14655253])

In [20]:
spotiy['song']

0                  Tell Me A Secret
1                     The Last Time
2                    Words Can Kill
3                     Hot 'n' Ready
4       Have You Ever Loved A Woman
                   ...             
3887                 Broken Hearted
3888                     Dragontown
3889                        At Last
3890                        Changes
3891                 Takin' It Back
Name: song, Length: 3892, dtype: object

Recommandendation funcation for music

In [21]:
from sympy import re


d=sorted(list(enumerate(songsim[3888])),reverse=True,key=lambda x:x[1])
d[0:5]
song=[]
for i in d[0:5]:
    song.append(spotiy.iloc[i[0]].song)

song

['Dragontown', 'Come Let Go', 'Chi', 'Come Back Baby', 'I Like It Like That']

In [22]:
#Example #spotiy[spotiy['song']=='Tell Me A Secret']
#spotiy[spotiy['song']=='Dragontown'].index[0]
def recommander(song_name):
    ids=spotiy[spotiy['song']==song_name].index[0]
    dis=sorted(list(enumerate(songsim[ids])),reverse=True,key=lambda x:x[1])  
    song=[]
    art=[]
    for i in dis[1:6]:
        s=spotiy.iloc[i[0]].song
        a=spotiy.iloc[i[0]].artist
        song.append(s)
        art.append(spotiy.iloc[i[0]].artist)
    return song,art

In [23]:
recommander("Dragontown")

(['Come Let Go',
  'Chi',
  'Come Back Baby',
  'I Like It Like That',
  'It Comes Back To You'],
 ['Xavier Rudd', 'Korn', 'Elton John', 'Kenny Loggins', 'Imagine Dragons'])

save the file as pickle to run in streamlit 

In [24]:
pickle.dump(songsim,open("music_recommendation.pkl","wb"))
pickle.dump(spotiy,open("df_music.pkl","wb"))